In [1]:
import pandas as pd
import numpy as np
import random
import os
import sys
sys.path.append('../..')
from modules.many_features import utils, constants, env
from modules.many_features.random_agent import RandomAgent
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline

C:\Users\User\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
SEED = 42
random.seed(SEED)
np.random.seed(SEED)

In [3]:
#df = pd.read_csv('../../data/anemia_synth_dataset_some_nans_unspecified_more_feats.csv')
#df = pd.read_csv('../../data/more_features/more_feats_new_labels_0.1.csv')
df = pd.read_csv('../../data/more_features/more_feats_correlated_noisy_6.csv')
df = df.fillna(-1)
df.head()

,hemoglobin,ferritin,ret_count,segmented_neutrophils,tibc,mcv,serum_iron,rbc,gender,creatinine,cholestrol,copper,ethanol,folate,glucose,hematocrit,tsat,label
0,12.322384,163.121429,4.121959,-1.00000,433.145097,100.147359,51.037057,3.691276,1,0.819482,147.693827,59.199141,41.958432,20.792161,101.383932,36.967153,11.782901,Unspecified anemia
1,8.298889,-1.000000,2.076950,-1.00000,483.617753,98.431076,-1.000000,2.529350,0,-1.000000,-1.000000,-1.000000,36.118322,-1.000000,-1.000000,24.896668,-1.000000,Hemolytic anemia
2,12.696391,3.393723,-1.000000,-1.00000,451.933132,79.486542,85.001345,4.791902,1,-1.000000,4.852168,89.831485,44.946238,0.965963,-1.000000,38.089174,18.808390,Iron deficiency anemia
3,12.705102,-1.000000,2.305379,-1.00000,-1.000000,81.057541,135.371313,4.702253,1,1.324140,32.717943,76.524319,-1.000000,27.439316,-1.000000,38.115305,-1.000000,Aplastic anemia
4,8.211543,29.622561,-1.000000,0.93619,479.914773,78.388480,-1.000000,3.142634,1,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,24.634629,-1.000000,Iron deficiency anemia


In [4]:
class_dict = constants.CLASS_DICT
df['label'] = df['label'].replace(class_dict)
X = df.iloc[:, 0:-1]
y = df.iloc[:, -1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=SEED)
X_train, y_train = np.array(X_train), np.array(y_train)
X_test, y_test = np.array(X_test), np.array(y_test)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((56000, 17), (14000, 17), (56000,), (14000,))

#### The Random Agent

In [5]:
%%time
rand_agent = RandomAgent(X_test[0:], y_test[0:])
test_df = rand_agent.test()
test_df.head()

Testing done.....
Wall time: 2min 30s


,episode_length,index,is_success,reward,terminated,trajectory,y_actual,y_pred
0,2.0,0.0,1.0,1.0,0.0,"[ret_count, Vitamin B12/Folate deficiency anemia]",1.0,1.0
1,1.0,1.0,0.0,-1.0,0.0,[No anemia],5.0,0.0
2,2.0,2.0,0.0,-1.0,0.0,"[segmented_neutrophils, Iron deficiency anemia]",3.0,4.0
3,1.0,3.0,0.0,-1.0,0.0,[Anemia of chronic disease],2.0,3.0
4,1.0,4.0,0.0,-1.0,0.0,[Anemia of chronic disease],0.0,3.0


In [6]:
# %%time
# rand_agent.test_sample(2)

In [7]:
# from datetime import datetime
# start = datetime.now()
# rand_agent.test_sample(2)
# end = datetime.now()
# duration = end-start
# duration

In [6]:
len(X_test), len(test_df)

(14000, 14000)

In [7]:
# test_df.to_csv('../../test_dfs/many_features/random_test_df_noisy_0.4.csv', index=False)

In [8]:
success_rate, success_df = utils.success_rate(test_df)
success_rate

12.428571428571429

In [9]:
avg_length, avg_return = utils.get_avg_length_reward(test_df)
avg_length, avg_return

(1.5399285714285715, -0.758)

In [10]:
acc, f1, roc_auc = utils.test(test_df['y_actual'], test_df['y_pred'])
acc, f1, roc_auc

(0.12428571428571429, 0.1232208623406614, 0.5027507499094873)

In [11]:
test_df.y_pred.unique()

array([1., 0., 4., 3., 2., 6., 7., 5.])